<a href="https://colab.research.google.com/github/mbarbetti/unifi-physics-lab3/blob/main/Calibrazione_oscilloscopio_con_ricerca_dei_picchi_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calibrazione temporale dell'oscilloscopio con ricerca dei picchi 
===
Lo scopo di questo programma è ricavare i coefficienti lineari per la conversione tra la misura del TDC implementato in LabView, che indicheremo come **unità aribitrarie** (*a.u.*), e i nanosecondi (*ns*). Il fit lineare ai picchi ottenuti dai segnali del TIME CALIBRATOR ci permetterà così di correggere le misure dei tempi da eventuali **errori sistematici** dovuti alla catena elettronica, all'oscilloscopio o a LabVIEW. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Prima di tutto dovete modificare il codice inserendo il nome del gruppo**

In [ ]:
## INSERIRE NUMERO DEL GRUPPO
group = "N2"   # esempio

Sfruttiamo il file di testo (".txt") per preparare un `DataFrame` Pandas, cioè una tabella organizzata per righe e colonne avente un set di funzioni utili per cercare, modificare, aggiungere o rimuovere elementi.

In [ ]:
file_path = f"https://raw.githubusercontent.com/mbarbetti/unifi-physics-lab3/main/data/2021/data_{group}.txt"
data = pd.read_csv (file_path, header = 4, delim_whitespace = True).drop([0])
data['QDC1'] = -data['QDC1'] * 1e9
data['QDC2'] = -data['QDC2'] * 1e9
data['TDC'] = data['TDC'] * 1e9
print(data)   # dataframe
print(min(data['QDC1'])," ",min(data['QDC1']))

Qui si riempiono degli istogrammi bi- e uni-dimensionali con i valori del QDC1 e QDC2 (rispettivamente muone ed elettrone). Le unità sono arbitrarie e sono moltiplicate per 1e09 per comodità di visualizzazione. Queste distribuzioni servono per selezionare gli eventi di fisica rispetto a quelli di calibrazione: guardando allo scatter plot **dovete decidere dove mettere i tagli per selezionare gli eventi corrispondenti al TIME CALIBRATOR**

In [ ]:
# Plot calib values
### SET THE QDC RANGES FOR TIME CALIBRATOR SIGNALS 
qdc1_max = 3.6
qdc2_max = 3.6

data_calib = data.query (f"(QDC1 < {qdc1_max}) & (QDC2 < {qdc2_max})")
qdc1 = data_calib['QDC1']
qdc2 = data_calib['QDC2']
qdc1_all = data['QDC1']
qdc2_all = data['QDC2']

plt.xlabel ("QDC1")
plt.ylabel ("QDC2")
plt.plot ([0,10], [qdc2_max, qdc2_max], color = "red", lw = 2)
plt.plot ([qdc1_max, qdc1_max], [0,10], color = "red", lw = 2)
plt.scatter (qdc1_all, qdc2_all, s=1)
plt.axis ([2,10,2,8])
plt.show()

nbin = 100

plt.xlabel ("QDC1")
plt.ylabel ("Entries")
plt.hist (qdc1_all, bins=nbin, range = (2,10), alpha = 0.75, label='QDC1')
plt.hist (qdc1, bins=nbin, range = (2,10), alpha = 0.75, label='QDC1')
plt.yscale("log")
plt.show()

plt.xlabel ("QDC2")
plt.ylabel ("Entries")
plt.hist (qdc2_all, bins=nbin, range = (2,10), alpha = 0.75, label='QDC2')
plt.hist (qdc2, bins=nbin, range = (2,10), alpha = 0.75, label='QDC2')
plt.yscale("log")
plt.show()

In [ ]:
### Prepare and fill histogram of time intervals
tdc = data_calib["TDC"].values

tmin = tdc.min()
tmax = tdc.max()
#nbins = tmax-tmin
nbins = 500

plt.figure (figsize = (10,8), dpi = 100)
plt.xlabel('Time interval [ns]')
plt.ylabel('Number of events')
plt.hist (data_calib['TDC'], bins=nbins, range = (tmin, tmax), label='Measured time intervals')
plt.yscale("linear")
plt.show()

#prepare date for peak finder 
nbins = int(data_calib['TDC'].max() + 1)
hist = np.histogram(tdc, bins=nbins, range=[0,nbins])
print(hist[0])
print(hist[1])

I tempi di riferimento per la calibrazione sono quelli prodotti dal **TIME CALIBRATOR**  che produce segnali di _start_ e _stop_ ogni 160 ns (o l'eventuale altro valore da voi impostato sul TIME CALIBRATOR) in una finestra temporale di circa 10.24 $\mu$s. Disponendo i tempi misurati dall'oscilloscopio in un istogramma, quello che ci aspettiamo di osservare è un "pettine" con 64 denti corrispondente ai 64 valori dei tempi generati dal TIME CALIBRATOR:

<center>$\mathcal{N}_{peaks} = \frac{\rm{finestra \, temporale}}{1/\rm{frequenza}} = \frac{10240 \, \rm{ns}}{160 \, \rm{ns}} = 64$</center>

In [ ]:
freq = 160   # time in ns
t_cal = np.arange (1,65) * freq
t_cal   # TIME CALIBRATOR values

La cella di codice sottostante permette di _fittare_ con una gaussiana un singolo picco contenuto nel "pettine".  
La funzione `gauss_fit` richiede i seguenti attributi:
* `sample`: un campione di misure dei tempi relativo a uno specifico picco;
* `bins`: numero di bins per costruire l'istogramma su cui fare il fit gaussiano;
* `range`: intervallo di valori scelto per l'istogramma su cui fare il fit gaussiano;
* `plot`: variabile booleana per abilitare (`True`) o meno (`False`) il plot del fit.

In [ ]:
from scipy.optimize import curve_fit

def gauss (x, mean, sigma, N):
  return (N / np.sqrt (2 * np.pi * sigma**2)) * np.exp (-0.5 * (x - mean)**2 / sigma**2)

def gauss_fit (sample, bins, range, plot = False):
  weights = np.ones_like (sample) / len (sample)
  entries, bin_edges = np.histogram (sample, bins = bins, range = range, weights = weights)
  #tot_entries = np.sum (entries)
  #print (tot_entries)

  bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2.
  min_bin = bin_centers.min()
  max_bin = bin_centers.max()
  width = max_bin - min_bin

  popt, pcov = curve_fit (gauss, bin_centers, entries, bounds = ([min_bin, 0.1*width, 0.], [max_bin, width, 1.]))
  perr = np.sqrt (np.diag (pcov))

  if plot:
    plt.figure (figsize = (8,5))
    plt.title ('Fit plot', fontsize = 14)
    plt.xlabel ('ADC counts [a.u.]', fontsize = 12)
    plt.ylabel ('Normalized entries', fontsize = 12)
    plt.hist (sample, bins = bins, range = range, weights = weights, color = 'royalblue', label = 'Data')
    plt.plot (bin_centers, gauss (bin_centers, popt[0], popt[1], popt[2]), color = 'red', linewidth = 2, label = 'Fit result')
    plt.legend (loc = 'upper left', fontsize = 10)
    plt.show()

  return popt, perr

La funzione `peak_study` permette di estrarre **sotto-campioni** di tempi misurati a partire dai dati contenuti in `sample`. Per far ciò, preso la cella di codice sottostante estrae il sotto-campione tra i valori `lower_edge` e `upper_edge` . Il campione così ottenuto viene quindi utilizzato per **caratterizzare** il picco. Se la _flag_ booleana `fit` è vera, allora il sotto-campione viene passato alla funzione `gauss_fit` che ne calcola centroide e deviazione standard (con relativi errori dovuti al fit). Se invece la flag booleana `fit` è falsa, allora `peak_study` ha come output la media e la deviazione standard dei valori contenuti nel sotto-campione (senza errori).

In [ ]:
def peak_study (sample, lower_edge, upper_edge, plot = False, fit = True):
#  mask = (sample > (time_val - bound)) & (sample < (time_val + bound))
  mask = (sample > lower_edge) & (sample < upper_edge)
  sub_sample = sample [mask]
  #print(sub_sample)

  min_val = sub_sample.min()
  max_val = sub_sample.max()
  width = max_val - min_val
  #print(min_val, max_val, width)

  bins  = 50
  _range = [min_val - 0.5 * width, max_val + 0.5 * width]

  if plot and not fit:
    print ('\n----- DATA PLOT -----')
    plt.figure (figsize = (8,5))
    plt.title  ('Time distribution', fontsize = 14)
    plt.xlabel ('Measured time', fontsize = 12)
    plt.ylabel ('Entries', fontsize = 12)
    plt.hist (sub_sample, bins = bins, range = _range, color = 'royalblue')
    plt.yscale("log")
    plt.show()

  if fit and plot:
    if plot: print ('\n----- FIT PLOT -----')
    results, errs = gauss_fit (sub_sample, bins, _range, plot = plot)
    mean, sigma, N = results
    mean_err, sigma_err, N_err = errs
  elif not fit:
    mean = np.mean (sub_sample)
    mean_err = 0.
    sigma = np.std (sub_sample)
    sigma_err = 0.

  mean_fit  = [mean, mean_err]
  sigma_fit = [sigma, sigma_err]
  return mean_fit, sigma_fit

Il codice che segue individua i picchi in un campione di dati. Attivando la _flag_ booleana `verbose` è possibile visualizzare i picchi individuati con sovrapposta la curva gaussiana ottenuta dal fit.

In [ ]:
### Algorithm for finding peaks in histogram of ADC counts
### The histogram is scanned, and an integration window is chosen around each peak to perform the gaussian fit
def findPeaks(hist, verbose=False):
    peaks = []
    RSEs = []
    starting_edge = 1
    edge = 2
    done = False
    headroom = 10

    while not done:
        integral = 0
        previous_integral = 0

        bin_content = hist[0][edge-1]
        next_bin_content = hist[0][edge]
        previous_integral = np.sum(hist[0][starting_edge:edge-1])
        integral = np.sum(hist[0][starting_edge:edge])

        if (integral > 0 and previous_integral < (edge-starting_edge) and starting_edge != edge -1) :
            starting_edge = edge - 1
            integral = np.sum(hist[0][starting_edge:edge])

        if (bin_content > next_bin_content  and integral > 10):
            for j in range(headroom):
                integral_temp = integral
                integral = np.sum(hist[0][starting_edge:edge+1+j])
                if (integral - integral_temp < 1) :
                    edge = edge + j 
                    break

            if (verbose):
              print("New peak found:")
              print(f"from {starting_edge} to {edge}")
              print(f"integral = {integral}") 

            mean, sigma = peak_study(tdc, starting_edge, edge, plot=True if verbose else False)
            weighted_avg = np.average(hist[1][starting_edge:edge], weights=hist[0][starting_edge:edge])
            peaks.append(mean[0])
            RSEs.append(mean[1])
            starting_edge = edge
        
        edge = edge +1

        if edge >= len(hist[0]) :
            done = True
        
    
    return peaks, RSEs

Chimando la funzione `findPeaks()` sui dati del TIME CALIBRATOR è possibile ottenere tutti i **centroidi** dei picchi che formano il "pettine".

In [ ]:
t_mis = list()
errs  = list()

#times = np.linspace (tmin, tmax, len(t_cal))
#print (times)

t_mis, errs = findPeaks(hist, True)

df = pd.DataFrame (np.c_ [t_cal, t_mis, errs], columns = ['Times', 'ADC counts', 'Uncertainties'])
df

A questo punto abbiamo tutto il necessario per procedere con il **fit lineare**!

In [ ]:
ones = np.ones_like (t_cal)

X = np.c_ [t_cal, ones]
V = np.diag (np.square (errs))
V_inv = np.linalg.inv (V)

B = np.linalg.inv (X.T @ V_inv @ X) @ X.T @ V_inv
theta = B @ t_mis
U = B @ V @ B.T
sigmas = np.diag(U)
sigmas = np.sqrt(sigmas)
print ('Results from least square fit:')
print (f'    Slope = {theta[0]:.6f} +/- {sigmas[0]:.6f} a.u./ns')
print (f'    Intercept = {theta[1]:.5f} +/- {sigmas[1]:.5f} a.u.')
print ('\nCorrelation matrix:')
print (U)

Verifichiamo la bontà del fit *plottando* i risultati ottenuti!

In [ ]:
plt.figure (figsize = (10, 7))
plt.title ('Time calibration', fontsize = 14)
plt.xlabel ('True time [ns]', fontsize = 12)
plt.ylabel ('Measured time [ns]', fontsize = 12)

t_fit = theta[0] * t_cal + theta[1]

plt.errorbar (t_cal, t_mis, yerr = errs, color = 'blue', fmt = 'v',
              markersize = 4, barsabove = True, capsize = 2, label = 'Data points')
plt.plot (t_cal, t_fit, color = 'red', linewidth = 1, label = 'Calibration fit')

plt.legend (loc = 'upper left', fontsize = 10)
plt.show()

L'ampia finestra considerata per i tempi non permette di apprezzare eventuali _disaccordi_ con quanto ottenuto dal fit. Per evidenziarli, riportiamo su grafico i cosiddetti **residui**:

<center>$\rm{residuals} = y_{\rm{true}} - y_{\rm{fit}}$</center>

In [ ]:
plt.figure (figsize = (10, 7))
plt.title ('Chi square residuals', fontsize = 14)
plt.xlabel ('ADC counts [a.u.]', fontsize = 12)
plt.ylabel ('Residuals', fontsize = 12)

residuals = t_mis - t_fit

plt.errorbar (t_mis, residuals, yerr = errs, color = 'blue', fmt = 'v',
              markersize = 4, barsabove = True, capsize = 2, label = 'Residuals [a.u.]')
#plt.plot (t_mis, 100 * residuals / np.array(t_mis), color = 'green', 
#          linewidth = 1, label = 'Residuals [%]')

plt.hlines (0, t_mis[0], t_mis[-1], color = 'red', linestyle='dashed')

plt.legend (loc = 'lower right', fontsize = 10)
plt.show()

Quindi, sfruttando la libreria `stats` di SciPy, calcoliamo il $\chi^2$ associato al fit:

In [ ]:
from scipy.stats import chi2

squares  = np.square (residuals/errs)
chi2_fit = np.sum (squares)
ndf = squares.size - 2
p_val = 1 - chi2.cdf (chi2_fit, ndf)

print ('Results from least square fit:')
print (f'    Slope = {theta[0]:.5f} a.u./ns')
print (f'    Intercept = {theta[1]:.5f} a.u.')
print (f'    chi2 = {chi2_fit:.3f}')
print (f'    ndf  = {ndf:.3f}')
print (f'    p-value = {100*p_val:.1f}%')

Infine, siamo in grado di calcolare il **fattore di conversione** da unità arbitrarie a nanosecondi, così come siamo in grado di **correggere** le misure dei tempi per tener conto del contributo sistematico introdotto dalla catena di misura. **Qual è l'errore sui due parametri `a` e `b`?**

In [ ]:
a = 1 / theta[0]
b = - theta[1] * conv_factor   #errore? 

print ('Results for muon lifetime study:')
print ('    a = {:.5f} ns/a.u.' . format (a))
print ('    b = {:.5f} ns' . format (b))

print ('\nt_corr = ({:.5f}) * t_mis + ({:.5f}) ns' . format (a, b))